In [39]:
import pandas as pd

df = pd.read_csv('../data/2018-11-metropolitan-street.csv').dropna(subset=['Longitude', 'Latitude'])

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90708 entries, 0 to 90707
Data columns (total 12 columns):
Crime ID                 72845 non-null object
Month                    90708 non-null object
Reported by              90708 non-null object
Falls within             90708 non-null object
Longitude                90708 non-null float64
Latitude                 90708 non-null float64
Location                 90708 non-null object
LSOA code                90708 non-null object
LSOA name                90708 non-null object
Crime type               90708 non-null object
Last outcome category    72845 non-null object
Context                  0 non-null float64
dtypes: float64(3), object(9)
memory usage: 9.0+ MB
None


In [40]:
df['Coordinate'] = df.apply(lambda x: (x['Longitude'], x['Latitude']), axis=1)

df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Coordinate
0,NaN,2018-11,Metropolitan Police Service,Metropolitan Police Service,0.134947,51.588063,On or near Mead Grove,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN,"(0.134947, 51.588063)"
1,NaN,2018-11,Metropolitan Police Service,Metropolitan Police Service,0.137065,51.583672,On or near Police Station,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN,"(0.13706500000000002, 51.583672)"
2,NaN,2018-11,Metropolitan Police Service,Metropolitan Police Service,0.133181,51.586719,On or near Rosehatch Avenue,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN,"(0.133181, 51.586719)"
3,NaN,2018-11,Metropolitan Police Service,Metropolitan Police Service,0.140192,51.582311,On or near Hatch Grove,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN,"(0.140192, 51.582311)"
4,NaN,2018-11,Metropolitan Police Service,Metropolitan Police Service,0.134947,51.588063,On or near Mead Grove,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN,"(0.134947, 51.588063)"


In [41]:
crime_sr = df.groupby(['Coordinate', 'Crime type']).size()

In [42]:
crime_freq = df.groupby('Coordinate').size()

print(crime_freq.min())
print(crime_freq.max())

1
210


In [43]:
features = []

last_coord = None
data = None

for (coord, crime_type), crime_count in crime_sr.iteritems():
    
    # Create a new data if it's the new location
    if coord != last_coord:
        if data is not None:
            features.append(data)
                
        # Re-initialise the data
        data = {
            'type': 'Feature',
            'properties': {
                'totalCrimeCount': 0,
                'crimeCountPerType': {}
            },
            'geometry': {
                'type': 'Point',
                'coordinates': []
            }
        }
        
    data['properties']['totalCrimeCount'] += crime_count
    data['properties']['crimeCountPerType'][crime_type] = crime_count
    data['geometry']['coordinates'] = [coord[0], coord[1]]
    
    last_coord = coord

In [44]:
import json

with open('./template.geojson', 'r') as template:
    template = json.load(template)
    template['features'] = features
    template['minCrimeCount'] = int(crime_freq.min())
    template['maxCrimeCount'] = int(crime_freq.max())

with open('2018-11-metropolitan-street.geojson', 'w') as output:
    json.dump(template, output)